In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import nest_asyncio
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
nest_asyncio.apply()

## Agent tools
Defining a tool to get the current exchange rate. We'll deploy this as an MCP
> Source: [Google's A2A repo](https://github.com/google/A2A/blob/main/samples/python/agents/langgraph/agent.py)

In [3]:
import httpx

def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
    """Use this to get current exchange rate.

    Args:
        currency_from: The currency to convert from (e.g., "USD").
        currency_to: The currency to convert to (e.g., "EUR").
        currency_date: The date for the exchange rate or "latest". Defaults to "latest".

    Returns:
        A dictionary containing the exchange rate data, or an error message if the request fails.
    """    
    try:
        response = httpx.get(
            f"https://api.frankfurter.app/{currency_date}",
            params={"from": currency_from, "to": currency_to},
        )
        response.raise_for_status()

        data = response.json()
        if "rates" not in data:
            return {"error": "Invalid API response format."}
        return data
    except httpx.HTTPError as e:
        return {"error": f"API request failed: {e}"}
    except ValueError:
        return {"error": "Invalid JSON response from API."}

In [4]:
get_exchange_rate(
    currency_from="SGD",
    currency_to="USD",
    currency_date="latest",
)

{'amount': 1.0, 'base': 'SGD', 'date': '2025-04-17', 'rates': {'USD': 0.76216}}

## Building the agent

In [5]:
from llama_index.llms.openai import OpenAI
from llama_index.core.agent.workflow import FunctionAgent, AgentStream
from llama_index.core.workflow import Context
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec

mcp_client = BasicMCPClient("http://localhost:3000/sse")
mcp_tool = McpToolSpec(client=mcp_client)
tools = await mcp_tool.to_tool_list_async()

agent = FunctionAgent(
    tools = tools,
    llm=OpenAI(model="gpt-4o-mini"),
)

In [13]:
ctx = Context(agent)
handler = agent.run("What is the exchange rate from SGD to USD?", ctx=ctx)
chat_history = []
async for event in handler.stream_events():
    if isinstance(event, AgentStream):
        print(event.delta, end="", flush=True)
    
response = await handler

The exchange rate from Singapore Dollar (SGD) to US Dollar (USD) is approximately 0.76216 USD for 1 SGD.

In [8]:
from IPython.display import display, Markdown

display(Markdown(str(response)))

The exchange rate from Singapore Dollar (SGD) to US Dollar (USD) is approximately 0.76216 USD for 1 SGD.

In [9]:
handler = agent.run("What is the exchange rate from SGD to EUR?", ctx=ctx)
response = await handler

In [10]:
str(response)

'The exchange rate from Singapore Dollar (SGD) to Euro (EUR) is approximately 0.67092 EUR for 1 SGD.'

Test persistence

In [11]:
handler = agent.run("What was the last thing I asked you?", ctx=ctx)
async for event in handler.stream_events():
    if isinstance(event, AgentStream):
        print(event.delta, end="", flush=True)

response = await handler

The exchange rate from Singapore Dollar (SGD) to Euro (EUR) is approximately 0.67092 EUR for 1 SGD.

In [15]:
from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)
ctx = Context(agent)
handler = agent.run("What is the exchange rate from USD to EURO?", ctx=ctx)
async for event in handler.stream_events():
    if isinstance(event, AgentStream):
        print(event.delta, end="", flush=True)
    elif isinstance(event, ToolCall):
        print("ToolCall event")
        print(event.tool_name)  # the tool name
        print(event.tool_kwargs)  # the tool kwargs

ToolCall event
get_exchange_rate
{'currency_from': 'USD', 'currency_to': 'EUR'}
ToolCall event
get_exchange_rate
{'currency_from': 'USD', 'currency_to': 'EUR'}
The exchange rate from USD to EURO is approximately 0.88028 EUR for 1 USD.

In [4]:
from llama_index.core.chat_engine import SimpleChatEngine
from llama_index.llms.openai import OpenAI

llm = OpenAI("gpt-4o-mini", temperature=0)
chat_engine = SimpleChatEngine.from_defaults(llm=llm)

In [5]:
full_response = ""
response = chat_engine.stream_chat("Hi!")
for token in response.response_gen:
    print(token, end="")
    full_response += token

Hello! How can I assist you today?